In [1]:
import pandas as pd
import os
import sys
from embeds import fix_corrupt, multi_inner_align, standardize
import numpy as np
from hypertools.tools import align
import torch

sys.path.append('..')
from rca.rca import run_rca

/Users/zhussain/miniconda3/envs/psychProbing_env/lib/python3.10/site-packages/hypertools/config.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


## Cognival

In [2]:
def read_txt(f) -> pd.DataFrame:
    """For reading the cognival data"""
    pulled = {}
    for line in f:
        word, *vec = line.split()
        pulled[word] = vec
    pulled = fix_corrupt(pulled)
    return pd.DataFrame(pulled).T.astype(float)

def read_individual_fmri(path: str) -> pd.DataFrame:
    dfs = []

    for f_name in os.listdir(path):

        if f_name.endswith('.txt'):
            with open(path + f_name, 'r') as f:
                dfs.append(read_txt(f))
        else:
            pass

    # Align indices
    dfs = list(multi_inner_align(dfs))

    return dfs

fmris_text_cognival = read_individual_fmri('../../data/fmri_text_cognival/')
fmris_speech_cognival = read_individual_fmri('../../data/fmri_speech_cognival/')

len(fmris_text_cognival), len(fmris_speech_cognival)

(8, 27)

In [3]:
# Standardize before hyper-aligning
fmris_text_cognival = [standardize(df) for df in fmris_text_cognival]
fmris_speech_cognival = [standardize(df) for df in fmris_speech_cognival]

# --- Hyper aligning individuals ---
def hyper_align(dfs: list) -> pd.DataFrame:
    df = np.mean(align(dfs, align='hyper'), axis=0)
    return pd.DataFrame(df, index=dfs[0].index)

fMRI_text_cognival = hyper_align(fmris_text_cognival)
fMRI_speech_cognival = hyper_align(fmris_speech_cognival)

## Denoised (Antonia)

In [4]:
def read_from_torch(f_path) -> pd.DataFrame:
    fmri_dict = torch.load(f_path, weights_only=False)
    voc, vecs = fmri_dict['dico'], fmri_dict['vectors'].numpy()
    return pd.DataFrame(vecs, index=voc, dtype=float)

denoise_path_template = '../../data/fmri_text_denoise/fMRI_text_denoise_{}d.pth'

fMRI_text_denoise_128d = read_from_torch(denoise_path_template.format(128))
fMRI_text_denoise_256d = read_from_torch(denoise_path_template.format(256))
fMRI_text_denoise_512d = read_from_torch(denoise_path_template.format(512))
fMRI_text_denoise_1024d = read_from_torch(denoise_path_template.format(1024))

fMRI_text_denoise_128d

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
Harry,-143.982681,51.265324,-73.016899,74.548111,141.569458,-3.107077,14.127753,-29.177664,12.766369,21.466639,...,-6.536313,-1.969234,6.133821,-0.842457,0.370705,2.955463,-4.847620,-1.390713,2.575084,1.202820
had,-127.148003,-417.956604,299.856567,-150.026901,-94.690819,93.597816,63.608559,41.971951,-9.625628,46.413219,...,-5.443430,7.407501,2.746191,-5.817649,-0.660896,2.467710,10.343082,-8.100626,-2.753611,7.861194
never,-725.864990,-410.511505,211.946350,-221.292572,-96.619041,424.433136,212.477356,1.393820,-152.869812,90.387222,...,17.369629,27.869184,-11.553320,-13.483357,10.268998,0.918990,-11.896314,-13.483791,19.849094,16.825212
believed,-2717.895020,-1213.610840,1656.710449,-414.832001,138.131638,981.171082,517.020569,-92.849586,16.785938,559.375793,...,52.605537,54.383560,-109.776634,-44.306526,-11.851886,32.966724,-47.805988,-70.414574,82.728691,16.483789
he,-348.185120,-86.909981,161.300644,425.559357,253.294922,29.610430,-20.238287,-30.480759,-27.076218,36.708912,...,5.568176,-2.478137,-9.667665,0.090169,7.073146,-6.388587,7.460380,-5.873054,17.383780,-3.726576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
grubby,5493.725586,-860.738159,96.581200,-114.270538,-524.003601,248.288269,201.176880,-826.146423,10.335138,271.975098,...,-12.817405,-53.413204,15.273238,11.075395,-32.439362,-51.257465,36.036182,-43.363522,1.828963,-52.249451
vault,5521.819336,-888.447327,73.344925,-55.472729,-534.191345,239.823303,245.571671,-894.114502,-169.258316,58.534554,...,-8.671092,-55.320320,-13.919680,21.021137,-20.458317,-27.349249,54.737080,-37.851021,0.523778,-30.322697
seven,5515.584473,-892.888733,57.494415,-48.431969,-528.394165,241.797775,261.311951,-903.200195,-216.163422,22.045902,...,-9.460995,-51.728687,-28.285343,16.569675,-13.407843,-11.184761,65.148331,-30.104380,-0.721720,-25.696573
hundred,5517.106934,-894.937744,60.638649,-49.630238,-528.597656,241.865143,261.439636,-898.800964,-215.590195,18.218752,...,-9.753736,-51.207165,-29.787228,12.147860,-8.493997,-7.011636,65.823349,-25.347975,-0.434391,-23.963270


In [5]:
# Finding best denoise dimensionality
to_compare = {
    'fMRI_text_cognival': fMRI_text_cognival.copy(),
    'fMRI_text_denoise_128d': fMRI_text_denoise_128d.copy(),
    'fMRI_text_denoise_256d': fMRI_text_denoise_256d.copy(),
    'fMRI_text_denoise_512d': fMRI_text_denoise_512d.copy(),
    'fMRI_text_denoise_1024d': fMRI_text_denoise_1024d.copy()
}

# Aligning for fair comparison
to_compare = dict(zip(to_compare.keys(), multi_inner_align(to_compare.values())))

# Standardizing
to_compare = {name: standardize(embed) for name, embed in to_compare.items()}

# Loading norm data
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, low_memory=False, compression='zip')
norms_meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col='norm')
norms

,frequency_lund,frequency_kucera,frequency_subtlexus,frequency_subtlexuk,frequency_blog_gimenes,frequency_twitter_gimenes,frequency_news_gimenes,frequency_written_cobuild,frequency_spoken_cobuild,context_diversity_subtlexus,...,person_vanarsdall,goals_vanarsdall,movement_vanarsdall,concreteness_vanarsdall,familiarity_vanarsdall,imageability_vanarsdall,familiarity_fear,aoa_fear,imageability_fear,sensory_experience_juhasz2013
word,,,,,,,,,,,,,,,,,,,,,
'em,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.3617,1.9138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'neath,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
're,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.9031,1.6335,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'shun,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'tis,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.4771,0.6021,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
shrick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.62,4.38,2.93,NaN
post office,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.79,3.07,5.29,NaN
fishing rod,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.29,3.38,5.64,NaN


In [6]:
norms = norms[[norm for norm in norms if 'glasgow' in norm]]
results = run_rca(to_compare, norms, norms_meta, n_jobs=10)
results

  0%|          | 0/5 [00:00<?, ?it/s]

fMRI_text_cognival:   0%|          | 0/9 [00:00<?, ?it/s]

                         norm  train_n   r2_mean     r2_sd check
2        concreteness_glasgow      303 -0.016066  0.016143  pass
1                 aoa_glasgow      303 -0.019051  0.047200  pass
3        imageability_glasgow      303 -0.020584  0.037134  pass
0         familiarity_glasgow      303 -0.020753  0.032926  pass
5  gender_association_glasgow      303 -0.023693  0.039578  pass
8           dominance_glasgow      303 -0.030578  0.035726  pass
4       semantic_size_glasgow      303 -0.038038  0.048776  pass
7             arousal_glasgow      303 -0.044966  0.046516  pass
6             valence_glasgow      303 -0.045542  0.041437  pass


fMRI_text_denoise_128d:   0%|          | 0/9 [00:00<?, ?it/s]

                         norm  train_n   r2_mean     r2_sd check
3        imageability_glasgow      303  0.025236  0.047290  pass
2        concreteness_glasgow      303  0.016987  0.033734  pass
4       semantic_size_glasgow      303 -0.009469  0.031474  pass
1                 aoa_glasgow      303 -0.020800  0.023296  pass
5  gender_association_glasgow      303 -0.026060  0.033672  pass
0         familiarity_glasgow      303 -0.030157  0.034449  pass
8           dominance_glasgow      303 -0.035331  0.033286  pass
7             arousal_glasgow      303 -0.044686  0.043067  pass
6             valence_glasgow      303 -0.054036  0.053545  pass


...

fMRI_text_denoise_512d:   0%|          | 0/9 [00:00<?, ?it/s]

                         norm  train_n   r2_mean     r2_sd check
3        imageability_glasgow      303  0.028134  0.053308  pass
2        concreteness_glasgow      303  0.007363  0.034053  pass
1                 aoa_glasgow      303 -0.019178  0.024625  pass
5  gender_association_glasgow      303 -0.026336  0.033699  pass
0         familiarity_glasgow      303 -0.029808  0.034258  pass
7             arousal_glasgow      303 -0.030602  0.029518  pass
4       semantic_size_glasgow      303 -0.038750  0.043465  pass
8           dominance_glasgow      303 -0.040585  0.036440  pass
6             valence_glasgow      303 -0.052029  0.051216  pass


fMRI_text_denoise_1024d:   0%|          | 0/9 [00:00<?, ?it/s]

                         norm  train_n   r2_mean     r2_sd check
1                 aoa_glasgow      303 -0.019867  0.024253  pass
2        concreteness_glasgow      303 -0.023564  0.060891  pass
4       semantic_size_glasgow      303 -0.026573  0.035677  pass
3        imageability_glasgow      303 -0.026823  0.073104  pass
7             arousal_glasgow      303 -0.030493  0.029113  pass
8           dominance_glasgow      303 -0.033547  0.036686  pass
0         familiarity_glasgow      303 -0.042031  0.036812  pass
5  gender_association_glasgow      303 -0.053244  0.059603  pass
6             valence_glasgow      303 -0.054900  0.053152  pass


,embed,embed_type,norm,train_n,test_n,p,r2_mean,r2_sd,check
0,fMRI_text_cognival,None,familiarity_glasgow,303,76,1000,-0.020753,0.032926,pass
1,fMRI_text_cognival,None,aoa_glasgow,303,76,1000,-0.019051,0.047200,pass
2,fMRI_text_cognival,None,concreteness_glasgow,303,76,1000,-0.016066,0.016143,pass
3,fMRI_text_cognival,None,imageability_glasgow,303,76,1000,-0.020584,0.037134,pass
4,fMRI_text_cognival,None,semantic_size_glasgow,303,76,1000,-0.038038,0.048776,pass
5,fMRI_text_cognival,None,gender_association_glasgow,303,76,1000,-0.023693,0.039578,pass
6,fMRI_text_cognival,None,valence_glasgow,303,76,1000,-0.045542,0.041437,pass
7,fMRI_text_cognival,None,arousal_glasgow,303,76,1000,-0.044966,0.046516,pass
8,fMRI_text_cognival,None,dominance_glasgow,303,76,1000,-0.030578,0.035726,pass
9,fMRI_text_denoise_128d,None,familiarity_glasgow,303,76,128,-0.030157,0.034449,pass


## Comparing

In [7]:
# Adding norm category
results['norm_category'] = (
    results['norm']
    .apply(lambda norm: norms_meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

results_avg = (
    results[['norm_category', 'embed', 'r2_mean']]
    .groupby(['norm_category', 'embed'], as_index=False).median()
    .dropna()
)

results_avg_piv = results_avg.pivot(columns='embed', index='norm_category', values='r2_mean')
results_avg_piv.round(2)

embed,fMRI_text_cognival,fMRI_text_denoise_1024d,fMRI_text_denoise_128d,fMRI_text_denoise_256d,fMRI_text_denoise_512d
norm_category,,,,,
age of acquisition,-0.02,-0.02,-0.02,-0.02,-0.02
arousal,-0.04,-0.03,-0.04,-0.04,-0.03
concreteness,-0.02,-0.02,0.02,-0.01,0.01
dominance,-0.03,-0.03,-0.04,-0.03,-0.04
familiarity,-0.02,-0.04,-0.03,-0.03,-0.03
imageability,-0.02,-0.03,0.03,-0.02,0.03
social/moral,-0.02,-0.05,-0.03,-0.03,-0.03
space/time/quantity,-0.04,-0.03,-0.01,-0.04,-0.04
valence,-0.05,-0.05,-0.05,-0.05,-0.05


In [8]:
# Finding the top-performing fmri_text_denoise
sorted_overall = results_avg_piv.mean().sort_values(ascending=False)
sorted_overall

embed
fMRI_text_denoise_128d    -0.019813
fMRI_text_denoise_512d    -0.022421
fMRI_text_cognival        -0.028808
fMRI_text_denoise_256d    -0.029800
fMRI_text_denoise_1024d   -0.034560
dtype: float64

# Saving

In [ ]:
# Subsetting to only the words in norms
to_pull = set(
    pd.read_csv('../../data/psychNorms/psychNorms.zip', index_col=0, low_memory=False, compression='zip').index
)
fMRI_text_cognival = fMRI_text_cognival.loc[fMRI_text_cognival.index.isin(to_pull)].astype(float)
fMRI_speech_cognival = fMRI_speech_cognival.loc[fMRI_speech_cognival.index.isin(to_pull)].astype(float)
fMRI_text_denoise = fMRI_text_denoise_128d.loc[fMRI_text_denoise_128d.index.isin(to_pull)].astype(float)

# Saving 
fMRI_text_cognival.to_csv('../../data/embeds/fMRI_text_cognival.csv')
fMRI_speech_cognival.to_csv('../../data/embeds/fMRI_speech_cognival.csv')
fMRI_text_denoise.to_csv('../../data/embeds/fMRI_text_denoise.csv')